# Input libraries

In [16]:
from IPython.core.interactiveshell import InteractiveShell
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg


from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

# Input data

### sa data

In [17]:
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa.to_crs("EPSG:2193")
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)

CPU times: user 146 ms, sys: 118 ms, total: 264 ms
Wall time: 545 ms


### pga data

In [19]:
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pga = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))

# Merge SA data with PGA events 

In [20]:
PGA_SA = gpd.sjoin(pga, sa, op="within")

/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  """Entry point for launching an IPython kernel.


CPU times: user 27min 37s, sys: 10min 14s, total: 37min 51s
Wall time: 46min 40s


# Calculate Ignition probability

In [21]:
def SA_Ignition_Probability(merge_data):
    merge_data['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+merge_data['gmv_PGA'] + \
                                            0.0000984+merge_data['PopDensity']+ \
                                            0.001523+merge_data['BldgFloo_1'])))
    merge_data['IgnProb'] = 1 - \
        np.power(1-merge_data['SA_IgnPrb'], + 1/merge_data['NumberBldg'])
    return merge_data

In [22]:
IgnitionData = SA_Ignition_Probability(PGA_SA)

# Save to file

In [23]:
IgnitionData.drop(['lon', 'lat', 'index_right', \
                   'BldgFloo_1', 'PopDensity', 'NumberBldg'], \
                  axis=1, \
                 inplace=True)

In [ ]:
p_event = Path('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_event')

for event in range(IgnitionData['event_id'].max()):
    try:
        print(f"event {event}")
        i_data = IgnitionData[IgnitionData['event_id']==event]
        i_data.to_file(str(p_event) + f"/event_{event}.shp")
    except:
        print(f"no event {event}")

event 0


/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  import sys


event 1
event 2
event 3
event 4
event 5
event 6
event 7
event 8
event 9
event 10
event 11
event 12
event 13
event 14
event 15
event 16
event 17
event 18
event 19
event 20
event 21
event 22
event 23
event 24
event 25
event 26
event 27
event 28
event 29
event 30
event 31
event 32
event 33
event 34
event 35
event 36
event 37
event 38
event 39
event 40
event 41
event 42
event 43
event 44
event 45
event 46
event 47
event 48
event 49
event 50
event 51
event 52
event 53
event 54
event 55
event 56
event 57
event 58
event 59
event 60
event 61
event 62
event 63
event 64
event 65
event 66
event 67
event 68
event 69
event 70
event 71
event 72
event 73
event 74
event 75
event 76
event 77
event 78
event 79
event 80
event 81
event 82
event 83
event 84
event 85
event 86
event 87
event 88
event 89
event 90
event 91
event 92
event 93
event 94
event 95
event 96
event 97
event 98
event 99
event 100
event 101
event 102
event 103
event 104
event 105
event 106
event 107
event 108
event 109
event 110
event 11

event 665
event 666
event 667
event 668
event 669
event 670
event 671
event 672
event 673
event 674
event 675
event 676
event 677
event 678
event 679
event 680
event 681
event 682
event 683
event 684
event 685
event 686
event 687
event 688
event 689
event 690
event 691
event 692
event 693
event 694
event 695
event 696
event 697
event 698
event 699
event 700
event 701
event 702
event 703
event 704
event 705
event 706
event 707
event 708
event 709
event 710
event 711
event 712
event 713
event 714
event 715
event 716
event 717
event 718
event 719
event 720
event 721
event 722
event 723
event 724
event 725
event 726
event 727
event 728
event 729
event 730
event 731
event 732
event 733
event 734
event 735
event 736
event 737
event 738
event 739
event 740
event 741
event 742
event 743
event 744
event 745
event 746
event 747
event 748
event 749
event 750
event 751
event 752
event 753
event 754
event 755
event 756
event 757
event 758
event 759
event 760
event 761
event 762
event 763
event 764


event 1443
event 1444
event 1445
event 1446
event 1447
event 1448
event 1449
event 1450
event 1451
event 1452
event 1453
event 1454
event 1455
event 1456
event 1457
event 1458
event 1459
event 1460
event 1461
event 1462
event 1463
event 1464
event 1465
event 1466
event 1467
event 1468
event 1469
event 1470
event 1471
event 1472
event 1473
event 1474
event 1475
event 1476
event 1477
event 1478
event 1479
event 1480
event 1481
event 1482
event 1483
event 1484
event 1485
event 1486
event 1487
event 1488
event 1489
event 1490
event 1491
event 1492
event 1493
event 1494
event 1495
event 1496
event 1497
event 1498
event 1499
event 1500
event 1501
event 1502
event 1503
event 1504
event 1505
event 1506
event 1507
event 1508
event 1509
event 1510
event 1511
event 1512
event 1513
event 1514
event 1515
event 1516
event 1517
event 1518
event 1519
event 1520
event 1521
event 1522
event 1523
event 1524
event 1525
event 1526
event 1527
event 1528
event 1529
event 1530
event 1531
event 1532
event 1533

...

event 5176
event 5177
event 5178
event 5179
event 5180
event 5181
event 5182
event 5183
event 5184
event 5185
event 5186
event 5187
event 5188
event 5189
event 5190
event 5191
event 5192
event 5193
event 5194
event 5195
event 5196
event 5197
event 5198
event 5199
event 5200
event 5201
event 5202
event 5203
event 5204
event 5205
event 5206
event 5207
event 5208
event 5209
event 5210
event 5211
event 5212
event 5213
event 5214
event 5215
event 5216
event 5217
event 5218
event 5219
event 5220
event 5221
event 5222
event 5223
event 5224
event 5225
event 5226
event 5227
event 5228
event 5229
event 5230
event 5231
event 5232
event 5233
event 5234
event 5235
event 5236
event 5237
event 5238
event 5239
event 5240
event 5241
event 5242
event 5243
event 5244
event 5245
event 5246
event 5247
event 5248
event 5249
event 5250
event 5251
event 5252
event 5253
event 5254
event 5255
event 5256
event 5257
event 5258
event 5259
event 5260
event 5261
event 5262
event 5263
event 5264
event 5265
event 5266

event 5924
event 5925
event 5926
event 5927
event 5928
event 5929
event 5930
event 5931
event 5932
event 5933
event 5934
event 5935
event 5936
event 5937
event 5938
event 5939
event 5940
event 5941
event 5942
event 5943
event 5944
event 5945
event 5946
event 5947
event 5948
event 5949
event 5950
event 5951
event 5952
event 5953
event 5954
event 5955
event 5956
event 5957
event 5958
event 5959
event 5960
event 5961
event 5962
event 5963
event 5964
event 5965
event 5966
event 5967
event 5968
event 5969
event 5970
event 5971
event 5972
event 5973
event 5974
event 5975
event 5976
event 5977
event 5978
event 5979
event 5980
event 5981
event 5982
event 5983
event 5984
event 5985
event 5986
event 5987
event 5988
event 5989
event 5990
event 5991
event 5992
event 5993
event 5994
event 5995
event 5996
event 5997
event 5998
event 5999
event 6000
event 6001
event 6002
event 6003
event 6004
event 6005
event 6006
event 6007
event 6008
event 6009
event 6010
event 6011
event 6012
event 6013
event 6014

event 6673
event 6674
event 6675
event 6676
event 6677
event 6678
event 6679
event 6680
event 6681
event 6682
event 6683
event 6684
event 6685
event 6686
event 6687
event 6688
event 6689
event 6690
event 6691
event 6692
event 6693
event 6694
event 6695
event 6696
event 6697
event 6698
event 6699
event 6700
event 6701
event 6702
event 6703
event 6704
event 6705
event 6706
event 6707
event 6708
event 6709
event 6710
event 6711
event 6712
event 6713
event 6714
event 6715
event 6716
event 6717
event 6718
event 6719
event 6720
event 6721
event 6722
event 6723
event 6724
event 6725
event 6726
event 6727
event 6728
event 6729
event 6730
event 6731
event 6732
event 6733
event 6734
event 6735
event 6736
event 6737
event 6738
event 6739
event 6740
event 6741
event 6742
event 6743
event 6744
event 6745
event 6746
event 6747
event 6748
event 6749
event 6750
event 6751
event 6752
event 6753
event 6754
event 6755
event 6756
event 6757
event 6758
event 6759
event 6760
event 6761
event 6762
event 6763

event 7175
event 7176
event 7177
event 7178
event 7179
event 7180
event 7181
event 7182
event 7183
event 7184
event 7185
event 7186
event 7187
event 7188
event 7189
event 7190
event 7191
event 7192
event 7193
event 7194
event 7195
event 7196
event 7197
event 7198
event 7199
event 7200
event 7201
event 7202
event 7203
event 7204
event 7205
event 7206
event 7207
event 7208
event 7209
event 7210
event 7211
event 7212
event 7213
event 7214
event 7215
event 7216
event 7217
event 7218
event 7219
event 7220
event 7221
event 7222
event 7223
event 7224
event 7225
event 7226
event 7227
event 7228
event 7229
event 7230
event 7231
event 7232
event 7233
event 7234
event 7235
event 7236
event 7237
event 7238
event 7239
event 7240
event 7241
event 7242
event 7243
event 7244
event 7245
event 7246
event 7247
event 7248
event 7249
event 7250
event 7251
event 7252
event 7253
event 7254
event 7255
event 7256
event 7257
event 7258
event 7259
event 7260
event 7261
event 7262
event 7263
event 7264
no event 7

event 7635
event 7636
event 7637
event 7638
event 7639
event 7640
event 7641
event 7642
event 7643
event 7644
event 7645
event 7646
event 7647
event 7648
event 7649
event 7650
event 7651
event 7652
event 7653
event 7654
event 7655
event 7656
event 7657
event 7658
event 7659
event 7660
event 7661
event 7662
event 7663
event 7664
event 7665
event 7666
event 7667
event 7668
event 7669
event 7670
event 7671
event 7672
event 7673
event 7674
event 7675
event 7676
event 7677
event 7678
event 7679
event 7680
event 7681
event 7682
event 7683
event 7684
event 7685
event 7686
event 7687
event 7688
event 7689
event 7690
event 7691
event 7692
event 7693
event 7694
event 7695
event 7696
event 7697
event 7698
event 7699
event 7700
event 7701
event 7702
event 7703
event 7704
event 7705
event 7706
event 7707
event 7708
event 7709
event 7710
event 7711
event 7712
event 7713
event 7714
event 7715
event 7716
event 7717
event 7718
event 7719
event 7720
event 7721
event 7722
event 7723
event 7724
event 7725